In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as Fb
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import random_split

d:\Programs\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)

In [3]:
device = set_device()

In [4]:
device

device(type='cuda', index=0)

In [5]:
# Step 1: Load the checkpoints
resnet_path_model_checkpoint = torch.load("resnet18model_best_checkpoint.pth.tar", map_location=device)
vgg_path_model_checkpoint = torch.load("vgg16model_best_checkpoint.pth.tar", map_location=device)
mobilenet_path_model_checkpoint = torch.load("mobilenetv2model_best_checkpoint.pth.tar", map_location=device)
densenet_path_model_checkpoint = torch.load("densenet121model_best_checkpoint.pth.tar", map_location=device)

In [6]:
# Step 2: Define the ensemble model
class EnsembleModel(nn.Module):
    def __init__(self, model1, model2, model3, model4):
        super(EnsembleModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model4 = model4

        # Define additional layers for ensemble if needed

    def forward(self, x):
        output1 = self.model1(x)
        output2 = self.model2(x)
        output3 = self.model3(x)
        output4 = self.model4(x)

        # Perform ensemble strategy (e.g., voting, averaging, etc.)
        ensemble_output = (output1 + output2 + output3 + output4) / 4

        return ensemble_output

In [7]:
#Load the weights into individual models
resnet18_model = models.resnet18().to(device)
num_ftrs = resnet18_model.fc.in_features
number_of_classes = 7
resnet18_model.fc = nn.Linear(num_ftrs, number_of_classes)
resnet18_model.load_state_dict(resnet_path_model_checkpoint['model'])


<All keys matched successfully>

In [8]:
vgg_model16_model = models.vgg16().to(device)
num_ftrs = vgg_model16_model.classifier[6].in_features
vgg_model16_model.classifier[6] = nn.Linear(num_ftrs, number_of_classes)
vgg_model16_model.load_state_dict(vgg_path_model_checkpoint['model'])

<All keys matched successfully>

In [9]:
mobilenet_v2_model = models.mobilenet_v2().to(device)
num_ftrs = mobilenet_v2_model.classifier[1].in_features
mobilenet_v2_model.classifier[1] = nn.Linear(num_ftrs, number_of_classes)
mobilenet_v2_model.load_state_dict(mobilenet_path_model_checkpoint['model'])

<All keys matched successfully>

In [10]:
densenet121_model = models.densenet121().to(device)
num_ftrs = densenet121_model.classifier.in_features
densenet121_model.classifier = nn.Linear(num_ftrs, number_of_classes)
densenet121_model.load_state_dict(densenet_path_model_checkpoint['model'])

<All keys matched successfully>

In [11]:
# Step 5: Create an instance of the ensemble model
ensemble_model = EnsembleModel(resnet18_model, vgg_model16_model, mobilenet_v2_model, densenet121_model)

In [12]:
ensemble_model = ensemble_model.to(device)

In [13]:
# Step 6: Set the ensemble model to evaluation mode
ensemble_model.eval()

EnsembleModel(
  (model1): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [18]:
!gdown 1WkOhU4QLzwLesCA1uJMfZfA_OY4ora9M

Downloading...
From (uriginal): https://drive.google.com/uc?id=1WkOhU4QLzwLesCA1uJMfZfA_OY4ora9M
From (redirected): https://drive.google.com/uc?id=1WkOhU4QLzwLesCA1uJMfZfA_OY4ora9M&confirm=t&uuid=35201065-12ca-47e8-b9e5-23e9fd230a0b
To: d:\SRH\Master Thesis\Notebooks\Training_Data.zip

  0%|          | 0.00/3.27G [00:00<?, ?B/s]
  0%|          | 524k/3.27G [00:00<10:37, 5.13MB/s]
  0%|          | 2.62M/3.27G [00:00<03:53, 14.0MB/s]
  0%|          | 5.24M/3.27G [00:00<03:19, 16.4MB/s]
  0%|          | 8.39M/3.27G [00:00<02:36, 20.9MB/s]
  0%|          | 12.1M/3.27G [00:00<02:09, 25.1MB/s]
  0%|          | 15.7M/3.27G [00:00<02:02, 26.5MB/s]
  1%|          | 19.4M/3.27G [00:00<01:53, 28.6MB/s]
  1%|          | 23.1M/3.27G [00:00<01:47, 30.3MB/s]
  1%|          | 26.7M/3.27G [00:01<01:42, 31.5MB/s]
  1%|          | 30.4M/3.27G [00:01<01:40, 32.3MB/s]
  1%|          | 34.1M/3.27G [00:01<01:38, 32.7MB/s]
  1%|          | 37.7M/3.27G [00:01<01:35, 33.7MB/s]
  1%|▏         | 41.4M/3.27G [00:0

In [19]:
zip_path = "Training_Data.zip"
extract_path = "data"

In [20]:
import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [21]:
import os

In [23]:
dataset_path = "data/Training_Data"
os.listdir(dataset_path)

['Computed Tomography (CT) Scan',
 'Endoscopy',
 'Histology',
 'Magnetic Resonance Imaging (MRI)',
 'Non-med',
 'Positron Emission Tomography (PET) Scan',
 'Ultrasound',
 'X-ray']

In [33]:
import shutil

shutil.rmtree("data/Training_Data/Non-med")

#!rm -rf data/Training_Data/Non-med

In [34]:
os.listdir(dataset_path)

['Computed Tomography (CT) Scan',
 'Endoscopy',
 'Histology',
 'Magnetic Resonance Imaging (MRI)',
 'Positron Emission Tomography (PET) Scan',
 'Ultrasound',
 'X-ray']

In [35]:
dataset = ImageFolder(dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor()
]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)

In [36]:
mean = 0.0
std = 0.0

for images, _ in dataloader:
    batch_mean = torch.mean(images, dim=(0, 2, 3))
    batch_std = torch.std(images, dim=(0, 2, 3))
    mean += batch_mean
    std += batch_std

mean /= len(dataloader)
std /= len(dataloader)

print("Mean:", mean)
print("Standard Deviation:", std)

Mean: tensor([0.4614, 0.4271, 0.4310])
Standard Deviation: tensor([0.3192, 0.3154, 0.3157])


In [37]:
dataset = ImageFolder(dataset_path, transform=transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a consistent size
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
]))
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=False)

In [38]:
from torch.utils.data import DataLoader, random_split
import random

In [39]:
# Get the length of the dataset
dataset_size = len(dataset)
print(dataset_size)

# Split the dataset into train and test
train_size = int(0.8 * dataset_size)
print(train_size)
test_size = dataset_size - train_size
print(test_size)

# Set the random seed for reproducibility
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)

# Create the train and test datasets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(random_seed))

# Create the train and test dataloaders
train_loader = DataLoader(train_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

9612
7689
1923


In [40]:
train_load_data = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle=True)
test_load_data = torch.utils.data.DataLoader(test_dataset, batch_size = 64, shuffle=False)

In [41]:
correct_predictions = 0
total_predictions = 0

In [42]:
with torch.no_grad():
    for images, labels in test_load_data:
        images = images.to(device)  # Move data to the device if using GPU
        labels = labels.to(device)

        # Forward pass through the ensemble model
        outputs = ensemble_model(images)

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        # Count correct predictions
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

In [43]:
# Calculate accuracy
accuracy = correct_predictions / total_predictions
print(f"Ensemble Model Accuracy on Validation Set: {accuracy * 100:.2f}%")

Ensemble Model Accuracy on Validation Set: 79.51%


In [44]:
with torch.no_grad():
    for images, labels in train_load_data:
        images = images.to(device)  # Move data to the device if using GPU
        labels = labels.to(device)

        # Forward pass through the ensemble model
        outputs = ensemble_model(images)

        # Get the predicted labels
        _, predicted = torch.max(outputs, 1)

        # Count correct predictions
        correct_predictions += (predicted == labels).sum().item()
        total_predictions += labels.size(0)

In [45]:
# Calculate accuracy
accuracy = correct_predictions / total_predictions
print(f"Ensemble Model Accuracy on Training Set: {accuracy * 100:.2f}%")

Ensemble Model Accuracy on Training Set: 85.67%


In [46]:
# Save the ensemble model
torch.save(ensemble_model, "enseble_model.pth")